In [5]:
import torch
from torch import nn
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

In [4]:
class SkipConnection(torch.nn.Module):
    def __init__(self, f_m, f_s=None):
        """
        Description
        """
        super().__init__()
        self.f_m = f_m
        self.f_s = f_s
        self.relu = nn.ReLU()
        
    def forward(self, X):
        """
        Description
        """
        if self.f_s is not None:
            return self.relu(self.f_s(X) + self.f_m(X))
        else:
            return self.relu(X + self.f_m(X))

In [ ]:
trainset = CIFAR10(root='./data', train=True,
                                        download=True)
trainloader = DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = CIFAR10(root='./data', train=False,
                                       download=True)
testloader = DataLoader(testset, batch_size=batch_size,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')